# Training Models on Colab

Install TensorFlow and Numpy

In [1]:
!pip install --upgrade pip
!pip install --upgrade protobuf 

In [2]:
%tensorflow_version 1.15
import tensorflow as tf
print(tf.__version__)

!pip install numpy

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2


Check GPU status

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7410 sha256=d4ee1b9366410fa0465b8d3413b2326e4182214a48609d987df7897f438fd860
  Stored in directory: /root/.cache/pip/wheels/79/c1/b2/b6fc2647f693a084da25e1d31328ab3dbb565cc58fea37e973
Successfully built gputil
Gen RAM Free: 12.6 GB  | Proc size: 461.4 MB
GPU RAM Free: 14968MB | Used: 111MB | Util   1% | Total 15079MB


Mount Google Drive folder

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

# change to working tensorflow directory on the drive
%cd '/content/gdrive/My Drive/models/'

Mounted at /content/gdrive
/content/gdrive/My Drive/models


Install protobuf and compile, install setup.py

In [6]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
%cd /content/gdrive/My Drive/models/research/
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/models/research/:/content/gdrive/My Drive/Tensorflow/models/research/slim'

!python setup.py build
!python setup.py install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 14 not upgraded.
Need to get 1,791 kB of archives.
After this operation, 7,807 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd

Check remaining GPU time

In [7]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.861469244758288


Start training

In [9]:
!pip install tf_slim
%cd /content/gdrive/My Drive/models/research/object_detection
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/models/research/:/content/gdrive/My Drive/Tensorflow/models/research/slim'

!python train.py --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_pets.config --logtostderr

Streaming output truncated to the last 5000 lines.
INFO:tensorflow:global step 3629: loss = 0.7002 (0.393 sec/step)
I1213 08:52:04.593868 140684867176320 learning.py:512] global step 3629: loss = 0.7002 (0.393 sec/step)
INFO:tensorflow:global step 3630: loss = 0.8822 (0.396 sec/step)
I1213 08:52:04.991373 140684867176320 learning.py:512] global step 3630: loss = 0.8822 (0.396 sec/step)
INFO:tensorflow:global step 3631: loss = 0.5843 (0.405 sec/step)
I1213 08:52:05.398155 140684867176320 learning.py:512] global step 3631: loss = 0.5843 (0.405 sec/step)
INFO:tensorflow:global step 3632: loss = 0.4542 (0.404 sec/step)
I1213 08:52:05.803565 140684867176320 learning.py:512] global step 3632: loss = 0.4542 (0.404 sec/step)
INFO:tensorflow:global step 3633: loss = 0.6091 (0.404 sec/step)
I1213 08:52:06.208626 140684867176320 learning.py:512] global step 3633: loss = 0.6091 (0.404 sec/step)
INFO:tensorflow:global step 3634: loss = 0.7136 (0.399 sec/step)
I1213 08:52:06.609166 140684867176320 l

Export inference graph

In [15]:
#  .ckpt needs to be updated every time to match last .ckpt generated
#  .config needs to be updated when changing model
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix training/model.ckpt-5880 --output_directory new_graph

Instructions for updating:
Please use `layer.__call__` method instead.
W1213 09:56:03.818013 139858699392896 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I1213 09:56:05.133437 139858699392896 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I1213 09:56:05.167932 139858699392896 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I1213 09:56:05.200891 139858699392896 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

Zip file in Google Drive

In [16]:
!zip -r model_graph.zip new_graph

  adding: new_graph/ (stored 0%)
  adding: new_graph/model.ckpt.data-00000-of-00001 (deflated 7%)
  adding: new_graph/model.ckpt.index (deflated 67%)
  adding: new_graph/checkpoint (deflated 42%)
  adding: new_graph/model.ckpt.meta (deflated 93%)
  adding: new_graph/frozen_inference_graph.pb (deflated 9%)
  adding: new_graph/saved_model/ (stored 0%)
  adding: new_graph/saved_model/variables/ (stored 0%)
  adding: new_graph/saved_model/saved_model.pb (deflated 9%)
  adding: new_graph/pipeline.config (deflated 69%)
